https://scikit-learn.org/stable/modules/outlier_detection.html#novelty-and-outlier-detection

In [15]:
import pandas as pd


df = pd.read_csv('creditcard_reduced.csv')
df = df.drop('Time', axis=1)

In [16]:
df.sample(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
24625,1.143483,-0.329553,1.043684,-0.684585,-0.850591,0.035692,-0.794091,0.137745,2.757769,-1.347032,...,-0.070740,0.294173,-0.023392,-0.003364,0.389308,-0.715786,0.081157,0.018347,11.85,0
6260,-0.368147,1.351672,-0.467387,-0.358998,0.354035,-1.290938,0.797592,0.001668,-0.334338,-0.434057,...,0.444008,1.454443,-0.058136,0.123844,-0.722501,-0.236673,0.262340,0.257413,0.75,0
12945,-1.220946,0.700466,1.758205,2.450490,1.727523,-0.232670,0.402678,0.069399,-0.852065,0.164364,...,0.051878,0.089313,-0.181744,-0.032759,0.480834,0.174415,-0.027145,0.063038,13.69,0
2071,2.161610,0.198279,-2.536953,0.413367,0.903409,-1.406622,0.921913,-0.542788,-0.048359,0.290875,...,0.247206,0.842037,-0.140362,0.813159,0.682784,0.692679,-0.133198,-0.088610,0.00,0
860,0.906154,-0.827662,0.961717,0.137440,-0.979868,0.747312,-0.907911,0.499396,0.843295,-0.184927,...,0.221442,0.542408,-0.048204,-0.228647,-0.048173,1.145123,-0.033547,0.011987,103.75,0


In [17]:
df.shape

(30200, 30)

In [18]:
df.Class.sum()

200

In [19]:
print(f"Fraud con proporcion : {(df.Class.mean() * 1000).round(2)} por mil")

Fraud con proporcion : 6.62 por mil


In [20]:
features = [ val for val in df.columns if val.startswith('V')] + ['Amount']
features

['V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'Amount']

- **SUPERVISADO** -> `.fit(X, y)`
- **NO SUPERVISADO** -> `.fit(X)`

In [52]:
from sklearn.ensemble import IsolationForest

model = IsolationForest(n_jobs=4, random_state=0)
model.fit(df[features])

IsolationForest(n_jobs=4, random_state=0)

In [42]:
anomaly_score = model.decision_function(df[features])

For each observation, tells whether or not (+1 or -1) it should be considered as an inlier according to the fitted model.

In [43]:
anomaly_decision = model.predict(df[features])

In [44]:
anomaly_score

array([ 0.13667704,  0.15205647,  0.08577238, ..., -0.11126125,
        0.09997113, -0.16330609])

In [45]:
anomaly_decision

array([ 1,  1,  1, ..., -1,  1, -1])

In [46]:
df['anomaly_score'] = anomaly_score
df['anomaly_decision'] = anomaly_decision

In [47]:
df.sample(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V23,V24,V25,V26,V27,V28,Amount,Class,anomaly_score,anomaly_decision
26672,1.922704,-0.189449,-0.853484,0.631061,-0.655126,-1.769673,0.204265,-0.298528,0.842122,-0.032188,...,0.418435,0.782394,-0.352520,-0.653211,-0.028091,-0.039004,34.15,0,0.138526,1
24299,-1.252910,1.438792,1.176722,2.986154,0.064634,0.384194,2.364079,-0.613115,-2.181216,1.487265,...,-0.135462,0.541304,-0.000075,-0.096486,-0.476921,-0.065900,226.75,0,0.088423,1
19006,-3.488118,-4.592867,-2.684845,-1.419319,3.195679,-3.474672,-0.406575,-0.340190,-1.098680,0.638975,...,-1.291204,-0.157456,-1.743794,1.038736,0.494541,-1.086781,137.31,0,-0.019791,-1
28737,-0.403178,1.086467,1.395406,-0.005039,-0.022363,-0.760986,0.643318,0.051909,-0.689086,-0.351603,...,0.007664,0.486945,-0.211999,0.041238,0.241247,0.090702,8.43,0,0.147339,1
26255,-0.995384,1.821862,1.136446,0.089549,0.001805,-0.966488,0.633903,-0.153002,1.375098,0.023533,...,0.022653,0.276134,-0.011349,0.044943,0.537790,0.275462,3.57,0,0.106595,1


In [48]:
df.anomaly_decision.value_counts()

 1    29235
-1      965
Name: anomaly_decision, dtype: int64

In [49]:
df.groupby(['Class', 'anomaly_decision']).anomaly_decision.count()

Class  anomaly_decision
0      -1                    808
        1                  29192
1      -1                    157
        1                     43
Name: anomaly_decision, dtype: int64

- 1 -> INLIER
- -1 -> OUTLIER

- https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSEt6826gASSDQeGe6Cigne1tb-dG2dhtexNEGGHvIgCRFOrCfyhLnlckmFUaQyHGEQc7s&usqp=CAU
- https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS9gjmzagy-xDWVok6HmlOFJBM4xZ3NxvqIKPR3njG-m2x11i3RzC2XAeok-jbrcimWXB8&usqp=CAU

https://miro.medium.com/v2/resize:fit:720/format:webp/1*03a4ae510dhKV6pVnqyXzA.png